In [1]:
from torch.utils.data import DataLoader

from weather.dataset import weather_dataset

dataset = weather_dataset()

train_set, test_set = dataset.split()
del dataset

batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

for s in train_loader:
    print(s)
    break


Found files:
['mpi_roof_2023a.csv', 'mpi_roof_2023b.csv']
Found a total of 52700 samples.


TypeError: 'DataLoader' object is not an iterator